In [2]:
# If alignment file other format, rewrite this function
def extractIDs(fastaAlignment):
    with open(fastaAlignment) as f:
        IDs = [line.split("|")[1] for line in f.readlines() if line.startswith(">")]
    return IDs

In [29]:
from apiFunctions import uniprotDownload, enaDownload
import pandas as pd

# Extract Ids from Fasta
ids = extractIDs("exampleAlignment.fasta")

# Infromation for API
fileName="proteins.tab"
query="id:"+"+OR+id:".join(ids)
format="tab"

# You can find possible columns on this page https://www.uniprot.org/help/uniprotkb_column_names
columns="id,entry name,genes,organism,comment(PTM),feature(SIGNAL),3d,database(EMBL),database(GenBank),database(GeneID)"

# Download File
uniprotDownload(fileName,query=query, format=format, columns=columns)

# Show with Pandas
proteinInfo = pd.read_csv(fileName, sep="\t").set_index("Entry").sort_index()
proteinInfo

,Entry name,Gene names,Organism,Post-translational modification,Signal peptide,3D,Cross-reference (EMBL),Cross-reference (GenBank),Cross-reference (GeneID)
Entry,,,,,,,,,
A0A024HLJ9,A0A024HLJ9_PSEKB,ppiA PKB_4073,Pseudomonas knackmussii (strain DSM 6978 / LMG...,NaN,"SIGNAL 1..22; /evidence=""ECO:0000256|RuleBase...",NaN,HG322950;,NaN,29894959;
A0A078LSA2,A0A078LSA2_9PSED,ppiA BN1079_01457,Pseudomonas saudiphocaensis,NaN,"SIGNAL 1..19; /evidence=""ECO:0000256|RuleBase...",NaN,CCSF01000001;,NaN,NaN
A0A084A040,A0A084A040_9GAMM,ppiA SRDD_23650,Serratia sp. DD3,NaN,"SIGNAL 1..24; /evidence=""ECO:0000256|RuleBase...",NaN,AYKS02000067;,NaN,NaN
A0A085JP37,A0A085JP37_9GAMM,ppiA GTPT_0406,Tatumella ptyseos ATCC 33301,NaN,"SIGNAL 1..24; /evidence=""ECO:0000256|RuleBase...",NaN,JMPR01000008;,NaN,NaN
A0A098FNZ9,A0A098FNZ9_9PSED,ppiA PXNS11_160019,Pseudomonas xanthomarina,NaN,"SIGNAL 1..20; /evidence=""ECO:0000256|RuleBase...",NaN,CCYE01000008;,NaN,NaN
...,...,...,...,...,...,...,...,...,...
S6ATG3,S6ATG3_PSERE,ppiA PCA10_17240,Pseudomonas resinovorans NBRC 106553,NaN,"SIGNAL 1..20; /evidence=""ECO:0000256|RuleBase...",NaN,AP013068;,NaN,41078826;
U3AMA7,U3AMA7_9VIBR,ppiA VAZ01S_010_00900,Vibrio azureus NBRC 104587,NaN,"SIGNAL 1..18; /evidence=""ECO:0000256|RuleBase...",NaN,BATL01000010;,NaN,NaN
W0HNX6,W0HNX6_9GAMM,ppiA Sant_0394,Sodalis praecaptivus,NaN,"SIGNAL 1..24; /evidence=""ECO:0000256|RuleBase...",NaN,CP006569;,NaN,NaN


In [24]:
# Download DNA sequence for first protein
uniprotId = ids[0]
fileName = uniprotId+".fasta"
enaId = proteinInfo.loc[uniprotId]["Cross-reference (EMBL)"].split(";")[0]

print(uniprotId)

#Download Fasta file
DNAFasta = enaDownload(fileName, enaId)
DNAFasta

A0A510XAS8
